# December 21, 2022
https://adventofcode.com/2022/day/21

In [4]:
import re
test_fn = "data/21_test.txt"
puz_fn = "data/21.txt"

In [42]:
class MonkeyTree:
    def __init__( self, fn ):
        self.dict = {}
        with open(fn, "r") as file:
            while True:
                line = file.readline().strip("\n")
                if not line:
                    break
                simian = Monkey(line, self)
                self.dict[ simian.name ] = simian

    def get( self, monkey ):
        return self.dict[monkey].value()

    def __str__( self ):
        return "\n".join( [str(monkey) for monkey in self.dict.values()] )
    def __repr__( self ):
        return str(self)
    
class Monkey:
    def __init__( self, line, tree ):
        self.tree = tree
        self.name, spec = line.split(": ")
        if re.fullmatch( r"\d+", spec):
            self.left = self.right = self.op = None
            self.number = int(spec)
        else:
            self.left, self.op, self.right = spec.split(" ")
            self.number = None

    def value(self):
        if self.number is not None:
            return self.number

        # We could make this cooler by overloading these operators
        if self.op == "+":
            return self.tree.dict[self.left].value() + self.tree.dict[self.right].value()
        if self.op == "-":
            return self.tree.dict[self.left].value() - self.tree.dict[self.right].value()
        if self.op == "*":
            return self.tree.dict[self.left].value() * self.tree.dict[self.right].value()
        if self.op == "/":
            return self.tree.dict[self.left].value() / self.tree.dict[self.right].value()


    def __str__(self):
        if self.left is None:
            return f'''{self.name} = {self.number}'''
        else:
            text = f'''{self.name} = {self.left} {self.op} {self.right} = '''
            if self.number is None:
                text += "?"
            else:
                text += self.number

        return text

    def __repr__(self):
        return str(self)

In [28]:
Monkey("root: pppw + sjmn", None)

root = pppw + sjmn = ?

In [29]:
Monkey("zczc: 2", None)

zczc = 2

### Part 1

In [43]:
jungle = MonkeyTree( test_fn )
jungle

root = pppw + sjmn = ?
dbpl = 5
cczh = sllz + lgvd = ?
zczc = 2
ptdq = humn - dvpt = ?
dvpt = 3
lfqf = 4
humn = 5
ljgn = 2
sjmn = drzm * dbpl = ?
sllz = 4
pppw = cczh / lfqf = ?
lgvd = ljgn * ptdq = ?
drzm = hmdt - zczc = ?
hmdt = 32

In [45]:
jungle.get("root")

152.0

In [46]:
jungle = MonkeyTree( puz_fn )
jungle.get("root")

169525884255464.0

### Part 2
Haha! I noticed that one monkey was named humn

In [73]:
# Update class to treat humn specially

class MonkeyTree:
    def __init__( self, fn ):
        self.dict = {}
        with open(fn, "r") as file:
            while True:
                line = file.readline().strip("\n")
                if not line:
                    break
                simian = Monkey(line, self)
                self.dict[ simian.name ] = simian

    def get( self, monkey ):
        return self.dict[monkey].value()

    def unknowns( self ):
        return "\n".join( [str(monkey) for monkey in self.dict.values() if monkey.value() is None])

    def __str__( self ):
        return "\n".join( [str(monkey) for monkey in self.dict.values()] )
    def __repr__( self ):
        return str(self)
    
class Monkey:
    def __init__( self, line, tree ):
        self.tree = tree
        self.name, spec = line.split(": ")
        if self.name == "humn":
             self.left = self.right = self.op = self.number = None
        elif re.fullmatch( r"\d+", spec):
            self.left = self.right = self.op = None
            self.number = int(spec)
        else:
            self.left, self.op, self.right = spec.split(" ")
            self.number = None
            if self.name == "root":
                self.op = "=="

    def value(self):
        if self.number is not None:
            return self.number

        # Human doesn't know how to answer
        if self.name == "humn":
            return None

        # We could make this cooler by overloading these operators
        lv = self.tree.dict[self.left].value()
        rv = self.tree.dict[self.right].value()
        if lv is None or rv is None:
            return None

        if self.op == "+":
            return lv + rv
        if self.op == "-":
            return lv - rv
        if self.op == "*":
            return lv * rv
        if self.op == "/":
            return lv / rv


    def __str__(self):
        if self.left is None:
            return f'''{self.name} = {self.number}'''
        else:
            lv = self.tree.dict[self.left].value()
            rv = self.tree.dict[self.right].value()
            me = self.number
            if lv is None:
                lv = "?"
            if rv is None:
                rv = "?"
            if self.number is None:
                me = "?"
            text = f'''{self.name} = {self.left} {self.op} {self.right} = {lv} {self.op} {rv} = {me}'''

        return text

    def __repr__(self):
        return str(self)

In [81]:
jungle = MonkeyTree( test_fn )
jungle.get("root")
print( jungle.unknowns() )

root = pppw == sjmn = ? == 150 = ?
cczh = sllz + lgvd = 4 + ? = ?
ptdq = humn - dvpt = ? - 3 = ?
humn = None
pppw = cczh / lfqf = ? / 4 = ?
lgvd = ljgn * ptdq = 2 * ? = ?


In [82]:
# 150 == (4+(2*(x-3)))/4
(((150 * 4) - 4) / 2) + 3

301.0

In [89]:
def solve_human( jungle ):
    root = jungle.dict["root"]
    lname = root.left
    rname = root.right
    lv = jungle.dict[lname].value()
    rv = jungle.dict[rname].value()

    if lv is None:
        cur = rv
        cmon = jungle.dict[lname]
    else:
        cur = lv
        cmon = jungle.dict[rname]

    # invert all the operations until we get back to humn
    while cmon.name != "humn":
        lmon = jungle.dict[ cmon.left ]
        rmon = jungle.dict[ cmon.right ]
        op, lv, rv = cmon.op, lmon.value(), rmon.value()
        if lv is None:
            if op == "+":
                cur = cur - rv
            elif op == "-":
                cur = cur + rv
            elif op == "*":
                cur = cur / rv
            else:
                cur = cur * rv
            # follow the unknown back to humn
            cmon = lmon
        else:
            if op == "+":
                cur = cur - lv
            elif op == "-": 
                cur = lv - cur
            elif op == "*":
                cur = cur / lv
            else:
                cur = lv / cur
            cmon = rmon

    return cur

In [90]:
solve_human( jungle )

301.0

In [91]:
jungle = MonkeyTree( puz_fn )
jungle.get("root")
solve_human( jungle )

3247317268284.0